In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex SDK: AutoML video object tracking model


## Installation

Install the latest (preview) version of Vertex SDK.


In [ ]:
! pip3 install -U google-cloud-aiplatform --user

Install the Google *cloud-storage* library as well.


In [ ]:
! pip3 install google-cloud-storage

### Restart the Kernel

Once you've installed the Vertex SDK and Google *cloud-storage*, you need to restart the notebook kernel so it can find the packages.


In [ ]:
import os

if not os.getenv("AUTORUN"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### GPU run-time

*Make sure you're running this notebook in a GPU runtime if you have that option. In Colab, select* **Runtime > Change Runtime Type > GPU**

### Set up your GCP project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a GCP project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project.](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [Enable the Vertex APIs and Compute Engine APIs.](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [Google Cloud SDK](https://cloud.google.com/sdk) is already installed in Google Cloud Notebooks.

5. Enter your project ID in the cell below. Then run the  cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.


In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend when possible, to choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You cannot use a Multi-Regional Storage bucket for training with Vertex. Not all regions provide support for all Vertex services. For the latest support per region, see [Region support for Vertex AI services](https://cloud.google.com/vertex-ai/docs/general/locations)


In [ ]:
REGION = "us-central1"  # @param {type: "string"}

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append onto the name of resources which will be created in this tutorial.


In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Authenticate your GCP account

**If you are using Google Cloud Notebooks**, your environment is already
authenticated. Skip this step.

*Note: If you are on an Vertex notebook and run the cell, the cell knows to skip executing the authentication steps.*


In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your Google Cloud account. This provides access
# to your Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Vertex, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this tutorial in a notebook locally, replace the string
    # below with the path to your service account key and run this cell to
    # authenticate your Google Cloud account.
    else:
        %env GOOGLE_APPLICATION_CREDENTIALS your_path_to_credentials.json

    # Log in to your account on Google Cloud
    ! gcloud auth login

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

This tutorial is designed to use training data that is in a public Cloud Storage bucket and a local Cloud Storage bucket for your batch predictions. You may alternatively use your own training data that you have stored in a local Cloud Storage bucket.

Set the name of your Cloud Storage bucket below. It must be unique across all Cloud Storage buckets.


In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.


In [ ]:
! gsutil mb -l $REGION gs://$BUCKET_NAME

Finally, validate access to your Cloud Storage bucket by examining its contents:


In [ ]:
! gsutil ls -al gs://$BUCKET_NAME

### Set up variables

Next, set up some variables used throughout the tutorial.
### Import libraries and define constants


#### Import Vertex SDK

Import the Vertex SDK into our Python environment.


In [ ]:
import os
import sys
import time

from google.cloud.aiplatform import gapic as aip
from google.protobuf import json_format
from google.protobuf.json_format import MessageToJson, ParseDict
from google.protobuf.struct_pb2 import Struct, Value

#### Vertex AI constants

Setup up the following constants for Vertex AI:

- `API_ENDPOINT`: The Vertex AI API service endpoint for dataset, model, job, pipeline and endpoint services.
- `PARENT`: The Vertex AI location root path for dataset, model and endpoint resources.


In [ ]:
# API Endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex AI location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

#### AutoML constants

Next, setup constants unique to AutoML video object tracking datasets and training:

- Dataset Schemas: Tells the managed dataset service which type of dataset it is.
- Data Labeling (Annotations) Schemas: Tells the managed dataset service how the data is labeled (annotated).
- Dataset Training Schemas: Tells the Vertex AI Pipelines service the task (e.g., classification) to train the model for.


In [ ]:
# Video Dataset type
VIDEO_SCHEMA = "google-cloud-aiplatform/schema/dataset/metadata/video_1.0.0.yaml"
# Video Labeling type
IMPORT_SCHEMA_VIDEO_OBJECT_TRACKING = "gs://google-cloud-aiplatform/schema/dataset/ioformat/video_object_tracking_io_format_1.0.0.yaml"
# Video Training task
TRAINING_VIDEO_OBJECT_TRACKING_SCHEMA = "gs://google-cloud-aiplatform/schema/trainingjob/definition/automl_video_object_tracking_1.0.0.yaml"

## Clients

The Vertex SDK works as a client/server model. On your side (the Python script) you will create a client that sends requests and receives responses from the server (Vertex).

You will use several clients in this tutorial, so set them all up upfront.

- Dataset Service for managed datasets.
- Model Service for managed models.
- Pipeline Service for training.
- Endpoint Service for deployment.
- Job Service for batch jobs and custom training.
- Prediction Service for serving. *Note*: Prediction has a different service endpoint.


In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_dataset_client():
    client = aip.DatasetServiceClient(client_options=client_options)
    return client


def create_model_client():
    client = aip.ModelServiceClient(client_options=client_options)
    return client


def create_pipeline_client():
    client = aip.PipelineServiceClient(client_options=client_options)
    return client


def create_endpoint_client():
    client = aip.EndpointServiceClient(client_options=client_options)
    return client


def create_prediction_client():
    client = aip.PredictionServiceClient(client_options=client_options)
    return client


def create_job_client():
    client = aip.JobServiceClient(client_options=client_options)
    return client


clients = {}
clients["dataset"] = create_dataset_client()
clients["model"] = create_model_client()
clients["pipeline"] = create_pipeline_client()
clients["endpoint"] = create_endpoint_client()
clients["prediction"] = create_prediction_client()
clients["job"] = create_job_client()

for client in clients.items():
    print(client)

In [ ]:
IMPORT_FILE = "gs://automl-video-demo-data/traffic_videos/traffic_videos_labels.csv"

In [ ]:
! gsutil cat $IMPORT_FILE | head -n 10

*Example output*:
```
gs://automl-video-demo-data/traffic_videos/highway_005.mp4,sedan,1565750291672021,11.933333,0.509205,0.594283,,,0.728737,0.760959,,
gs://automl-video-demo-data/traffic_videos/highway_005.mp4,pickup_suv_van,1565750291672171,17.566666,0.761241,0.498466,,,0.948839,0.668524,,
gs://automl-video-demo-data/traffic_videos/highway_005.mp4,pickup_suv_van,1565750291672223,20.433333,0.000000,0.465235,,,0.142638,0.665644,,
gs://automl-video-demo-data/traffic_videos/highway_005.mp4,pickup_suv_van,1565750291672347,25.766666,0.486523,0.592331,,,0.720611,0.776687,,
gs://automl-video-demo-data/traffic_videos/highway_005.mp4,pickup_suv_van,1565750291672575,28.966666,0.578534,0.652778,,,0.828647,0.862967,,
gs://automl-video-demo-data/traffic_videos/highway_005.mp4,pickup_suv_van,1565750291672549,28.966666,0.000000,0.518571,,,0.148841,0.737677,,
gs://automl-video-demo-data/traffic_videos/highway_005.mp4,pickup_suv_van,1565750291672599,28.966666,0.106979,0.458078,,,0.377877,0.678937,,
gs://automl-video-demo-data/traffic_videos/highway_005.mp4,pickup_suv_van,1565715798494273,32.466666,0.333083,0.485473,,,0.542722,0.647774,,
gs://automl-video-demo-data/traffic_videos/highway_005.mp4,sedan,1565715798494439,36.433333,0.935638,0.564839,,,1.000000,0.672182,,
gs://automl-video-demo-data/traffic_videos/highway_005.mp4,pickup_suv_van,1565715798494381,36.433333,0.000000,0.455703,,,0.164878,0.660083,,
```


## Create a dataset


### [projects.locations.datasets.create](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.datasets/create)


#### Request


In [ ]:
DATA_SCHEMA = VIDEO_SCHEMA

dataset = {
    "display_name": "traffic_" + TIMESTAMP,
    "metadata_schema_uri": "gs://" + DATA_SCHEMA,
}

print(
    MessageToJson(
        aip.CreateDatasetRequest(parent=PARENT, dataset=dataset).__dict__["_pb"]
    )
)

*Example output*:
```
{
  "parent": "projects/migration-ucaip-training/locations/us-central1",
  "dataset": {
    "displayName": "traffic_20210310013516",
    "metadataSchemaUri": "gs://google-cloud-aiplatform/schema/dataset/metadata/video_1.0.0.yaml"
  }
}
```


#### Call


In [ ]:
request = clients["dataset"].create_dataset(parent=PARENT, dataset=dataset)

#### Response


In [ ]:
result = request.result()

print(MessageToJson(result.__dict__["_pb"]))

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/datasets/7534187925055995904",
  "displayName": "traffic_20210310013516",
  "metadataSchemaUri": "gs://google-cloud-aiplatform/schema/dataset/metadata/video_1.0.0.yaml",
  "labels": {
    "aiplatform.googleapis.com/dataset_metadata_schema": "VIDEO"
  },
  "metadata": {
    "dataItemSchemaUri": "gs://google-cloud-aiplatform/schema/dataset/dataitem/video_1.0.0.yaml"
  }
}
```


In [ ]:
# The full unique ID for the dataset
dataset_id = result.name
# The short numeric ID for the dataset
dataset_short_id = dataset_id.split("/")[-1]

print(dataset_id)

### [projects.locations.datasets.import](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.datasets/import)


#### Request


In [ ]:
LABEL_SCHEMA = IMPORT_SCHEMA_VIDEO_OBJECT_TRACKING

import_config = {
    "gcs_source": {"uris": [IMPORT_FILE]},
    "import_schema_uri": LABEL_SCHEMA,
}

print(
    MessageToJson(
        aip.ImportDataRequest(
            name=dataset_short_id, import_configs=[import_config]
        ).__dict__["_pb"]
    )
)

*Example output*:
```
{
  "name": "7534187925055995904",
  "importConfigs": [
    {
      "gcsSource": {
        "uris": [
          "gs://automl-video-demo-data/traffic_videos/traffic_videos_labels.csv"
        ]
      },
      "importSchemaUri": "gs://google-cloud-aiplatform/schema/dataset/ioformat/video_object_tracking_io_format_1.0.0.yaml"
    }
  ]
}
```


#### Call


In [ ]:
request = clients["dataset"].import_data(
    name=dataset_id, import_configs=[import_config]
)

#### Response


In [ ]:
result = request.result()

print(MessageToJson(result.__dict__["_pb"]))

*Example output*:
```
{}
```


## Train a model


### [projects.locations.trainingPipelines.create](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.trainingPipelines/create)


#### Request


In [ ]:
TRAINING_SCHEMA = TRAINING_VIDEO_OBJECT_TRACKING_SCHEMA

task = Value(struct_value=Struct(fields={"model_type": Value(string_value="CLOUD")}))

training_pipeline = {
    "display_name": "traffic_" + TIMESTAMP,
    "training_task_definition": TRAINING_SCHEMA,
    "training_task_inputs": task,
    "input_data_config": {
        "dataset_id": dataset_short_id,
        "fraction_split": {"training_fraction": 0.8, "test_fraction": 0.2},
    },
    "model_to_upload": {"display_name": "traffic_" + TIMESTAMP},
}

print(
    MessageToJson(
        aip.CreateTrainingPipelineRequest(
            parent=PARENT, training_pipeline=training_pipeline
        ).__dict__["_pb"]
    )
)

*Example output*:
```
{
  "parent": "projects/migration-ucaip-training/locations/us-central1",
  "trainingPipeline": {
    "displayName": "traffic_20210310013516",
    "inputDataConfig": {
      "datasetId": "7534187925055995904",
      "fractionSplit": {
        "trainingFraction": 0.8,
        "testFraction": 0.2
      }
    },
    "trainingTaskDefinition": "gs://google-cloud-aiplatform/schema/trainingjob/definition/automl_video_object_tracking_1.0.0.yaml",
    "trainingTaskInputs": {
      "model_type": "CLOUD"
    },
    "modelToUpload": {
      "displayName": "traffic_20210310013516"
    }
  }
}
```


#### Call


In [ ]:
request = clients["pipeline"].create_training_pipeline(
    parent=PARENT, training_pipeline=training_pipeline
)

#### Response


In [ ]:
print(MessageToJson(request.__dict__["_pb"]))

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/trainingPipelines/4612961451915608064",
  "displayName": "traffic_20210310013516",
  "inputDataConfig": {
    "datasetId": "7534187925055995904",
    "fractionSplit": {
      "trainingFraction": 0.8,
      "testFraction": 0.2
    }
  },
  "trainingTaskDefinition": "gs://google-cloud-aiplatform/schema/trainingjob/definition/automl_video_object_tracking_1.0.0.yaml",
  "trainingTaskInputs": {
    "modelType": "CLOUD"
  },
  "modelToUpload": {
    "displayName": "traffic_20210310013516"
  },
  "state": "PIPELINE_STATE_PENDING",
  "createTime": "2021-03-10T13:09:36.473816Z",
  "updateTime": "2021-03-10T13:09:36.473816Z"
}
```


In [ ]:
# The full unique ID for the training pipeline
training_pipeline_id = request.name
# The short numeric ID for the training pipeline
training_pipeline_short_id = training_pipeline_id.split("/")[-1]

print(training_pipeline_id)

### [projects.locations.trainingPipelines.get](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.trainingPipelines/get)


#### Call


In [ ]:
request = clients["pipeline"].get_training_pipeline(name=training_pipeline_id)

#### Response


In [ ]:
print(MessageToJson(request.__dict__["_pb"]))

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/trainingPipelines/4612961451915608064",
  "displayName": "traffic_20210310013516",
  "inputDataConfig": {
    "datasetId": "7534187925055995904",
    "fractionSplit": {
      "trainingFraction": 0.8,
      "testFraction": 0.2
    }
  },
  "trainingTaskDefinition": "gs://google-cloud-aiplatform/schema/trainingjob/definition/automl_video_object_tracking_1.0.0.yaml",
  "trainingTaskInputs": {
    "modelType": "CLOUD"
  },
  "modelToUpload": {
    "displayName": "traffic_20210310013516"
  },
  "state": "PIPELINE_STATE_PENDING",
  "createTime": "2021-03-10T13:09:36.473816Z",
  "updateTime": "2021-03-10T13:09:36.473816Z"
}
```


In [ ]:
while True:
    response = clients["pipeline"].get_training_pipeline(name=training_pipeline_id)
    if response.state != aip.PipelineState.PIPELINE_STATE_SUCCEEDED:
        print("Training job has not completed:", response.state)
        model_to_deploy_name = None
        if response.state == aip.PipelineState.PIPELINE_STATE_FAILED:
            break
    else:
        model_id = response.model_to_upload.name
        print("Training Time:", response.end_time - response.start_time)
        break
    time.sleep(60)

print(model_id)

## Evaluate the model


### [projects.locations.models.evaluations.list](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.models.evaluations/list)


#### Call


In [ ]:
request = clients["model"].list_model_evaluations(parent=model_id)

#### Response


In [ ]:
import json

model_evaluations = [json.loads(MessageToJson(me.__dict__["_pb"])) for me in request]
# The evaluation slice
evaluation_slice = request.model_evaluations[0].name

print(json.dumps(model_evaluations, indent=2))

*Example output*:
```
[
  {
    "name": "projects/116273516712/locations/us-central1/models/6125898247828406272/evaluations/305090287452028928",
    "metricsSchemaUri": "gs://google-cloud-aiplatform/schema/modelevaluation/video_object_tracking_metrics_1.0.0.yaml",
    "metrics": {
      "boundingBoxMetrics": [
        {
          "meanAveragePrecision": 0.34263912,
          "iouThreshold": 0.5,
          "confidenceMetrics": [
            {
              "precision": 0.36842105,
              "recall": 1.0,
              "f1Score": 0.53846157
            },
            {
              "precision": 0.088,
              "confidenceThreshold": 0.032954127,
              "recall": 0.16541353,
              "f1Score": 0.11488251
            },
            {
              "precision": 0.08835341,
              "confidenceThreshold": 0.035069585,
              "recall": 0.16541353,
              "f1Score": 0.11518325
            },
            {
              "precision": 0.088709675,
              "recall": 0.16541353,
              "confidenceThreshold": 0.036181003,
              "f1Score": 0.115485564
            },
            {
              "recall": 0.16541353,
              "f1Score": 0.11578947,
              "confidenceThreshold": 0.037186295,
              "precision": 0.08906882
            },
            {
              "recall": 0.16541353,
              "precision": 0.08943089,
              "confidenceThreshold": 0.038205147,
              "f1Score": 0.116094984
            },
            
            # REMOVED FOR BREVITY
            {
              "recall": 0.007518797,
              "precision": 1.0,
              "confidenceThreshold": 0.66486305,
              "f1Score": 0.014925373
            },
            {
              "precision": 1.0,
              "confidenceThreshold": 1.0
            }
          ]
        }
      ],
      "boundingBoxMeanAveragePrecision": 0.34263912
    },
    "createTime": "2021-03-10T14:18:31.880535Z",
    "sliceDimensions": [
      "annotationSpec"
    ]
  }
]
```


### [projects.locations.models.evaluations.get](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.models.evaluations/get)


#### Call


In [ ]:
request = clients["model"].get_model_evaluation(name=evaluation_slice)

#### Response


In [ ]:
print(MessageToJson(request.__dict__["_pb"]))

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/models/6125898247828406272/evaluations/305090287452028928",
  "metricsSchemaUri": "gs://google-cloud-aiplatform/schema/modelevaluation/video_object_tracking_metrics_1.0.0.yaml",
  "metrics": {
    "boundingBoxMetrics": [
      {
        "confidenceMetrics": [
          {
            "recall": 1.0,
            "precision": 0.36842105,
            "f1Score": 0.53846157
          },
          {
            "recall": 0.16541353,
            "precision": 0.088,
            "f1Score": 0.11488251,
            "confidenceThreshold": 0.032954127
          },
          
          # REMOVED FOR BREVITY
          
          {
            "confidenceThreshold": 1.0,
            "precision": 1.0
          }
        ],
        "meanAveragePrecision": 0.34263912,
        "iouThreshold": 0.5
      }
    ],
    "boundingBoxMeanAveragePrecision": 0.34263912
  },
  "createTime": "2021-03-10T14:18:31.880535Z",
  "sliceDimensions": [
    "annotationSpec"
  ]
}
```


## Make batch predictions


### Prepare batch prediction data


In [ ]:
test_items = ! gsutil cat $IMPORT_FILE | head -n25

cols_1 = test_items[0].split(",")
cols_2 = test_items[-1].split(",")

if len(cols_1) > 12:
    test_item_1 = str(cols_1[1])
    test_item_2 = str(cols_2[1])
    test_label_1 = str(cols_1[5:])
    test_label_2 = str(cols_2[5:])
else:
    test_item_1 = str(cols_1[0])
    test_item_2 = str(cols_2[0])
    test_label_1 = str(cols_1[4:])
    test_label_2 = str(cols_2[4:])


print(test_item_1, test_label_1)
print(test_item_2, test_label_2)

*Example output*:
```
gs://automl-video-demo-data/traffic_videos/highway_005.mp4 ['0.509205', '0.594283', '', '', '0.728737', '0.760959', '', '']
gs://automl-video-demo-data/traffic_videos/highway_006.mp4 ['0.621857', '0.561570', '', '', '0.825726', '0.699151', '', '']
```


### Make the batch input file

Let's now make a batch input file, which you store in your local Cloud Storage bucket. The batch input file can be either CSV or JSONL. You will use JSONL in this tutorial. For JSONL file, you make one dictionary entry per line for each video. The dictionary contains the key/value pairs:

- `content`: The Cloud Storage path to the video.
- `mimeType`: The content type. In our example, it is an `avi` file.
- `timeSegmentStart`: The start timestamp in the video to do prediction on. *Note*, the timestamp must be specified as a string and followed by s (second), m (minute) or h (hour).
- `timeSegmentEnd`: The end timestamp in the video to do prediction on.


In [ ]:
import json

import tensorflow as tf

gcs_input_uri = "gs://" + BUCKET_NAME + "/test.jsonl"
with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
    data = {
        "content": test_item_1,
        "mimeType": "video/avi",
        "timeSegmentStart": "0.0s",
        "timeSegmentEnd": "inf",
    }
    f.write(json.dumps(data) + "\n")
    data = {
        "content": test_item_2,
        "mimeType": "video/avi",
        "timeSegmentStart": "0.0s",
        "timeSegmentEnd": "inf",
    }
    f.write(json.dumps(data) + "\n")

print(gcs_input_uri)
!gsutil cat $gcs_input_uri

*Example output*:
```
gs://migration-ucaip-trainingaip-20210310013516/test.jsonl
{"content": "gs://automl-video-demo-data/traffic_videos/highway_005.mp4", "mimeType": "video/avi", "timeSegmentStart": "0.0s", "timeSegmentEnd": "inf"}
{"content": "gs://automl-video-demo-data/traffic_videos/highway_006.mp4", "mimeType": "video/avi", "timeSegmentStart": "0.0s", "timeSegmentEnd": "inf"}
```


### [projects.locations.batchPredictionJobs.create](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.batchPredictionJobs/create)


#### Request


In [ ]:
batch_prediction_job = {
    "display_name": "traffic_" + TIMESTAMP,
    # Format: 'projects/{project}/locations/{location}/models/{model_id}'
    "model": model_id,
    "model_parameters": json_format.ParseDict(
        {"confidenceThreshold": 0.5, "maxPredictions": 2}, Value()
    ),
    "input_config": {
        "instances_format": "jsonl",
        "gcs_source": {"uris": [gcs_input_uri]},
    },
    "output_config": {
        "predictions_format": "jsonl",
        "gcs_destination": {
            "output_uri_prefix": "gs://" + f"{BUCKET_NAME}/batch_output/"
        },
    },
    "dedicated_resources": {
        "machine_spec": {"machine_type": "n1-standard-4", "accelerator_count": 0},
        "starting_replica_count": 1,
        "max_replica_count": 1,
    },
}

print(
    MessageToJson(
        aip.CreateBatchPredictionJobRequest(
            parent=PARENT, batch_prediction_job=batch_prediction_job
        ).__dict__["_pb"]
    )
)

*Example output*:
```
{
  "parent": "projects/migration-ucaip-training/locations/us-central1",
  "batchPredictionJob": {
    "displayName": "traffic_20210310013516",
    "model": "projects/116273516712/locations/us-central1/models/6125898247828406272",
    "inputConfig": {
      "instancesFormat": "jsonl",
      "gcsSource": {
        "uris": [
          "gs://migration-ucaip-trainingaip-20210310013516/test.jsonl"
        ]
      }
    },
    "modelParameters": {
      "maxPredictions": 2.0,
      "confidenceThreshold": 0.5
    },
    "outputConfig": {
      "predictionsFormat": "jsonl",
      "gcsDestination": {
        "outputUriPrefix": "gs://migration-ucaip-trainingaip-20210310013516/batch_output/"
      }
    },
    "dedicatedResources": {
      "machineSpec": {
        "machineType": "n1-standard-4"
      },
      "startingReplicaCount": 1,
      "maxReplicaCount": 1
    }
  }
}
```


#### Call


In [ ]:
request = clients["job"].create_batch_prediction_job(
    parent=PARENT, batch_prediction_job=batch_prediction_job
)

#### Response


In [ ]:
print(MessageToJson(request.__dict__["_pb"]))

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/batchPredictionJobs/6806214470445039616",
  "displayName": "traffic_20210310013516",
  "model": "projects/116273516712/locations/us-central1/models/6125898247828406272",
  "inputConfig": {
    "instancesFormat": "jsonl",
    "gcsSource": {
      "uris": [
        "gs://migration-ucaip-trainingaip-20210310013516/test.jsonl"
      ]
    }
  },
  "modelParameters": {
    "maxPredictions": 2.0,
    "confidenceThreshold": 0.5
  },
  "outputConfig": {
    "predictionsFormat": "jsonl",
    "gcsDestination": {
      "outputUriPrefix": "gs://migration-ucaip-trainingaip-20210310013516/batch_output/"
    }
  },
  "state": "JOB_STATE_PENDING",
  "completionStats": {
    "incompleteCount": "-1"
  },
  "createTime": "2021-03-10T14:23:59.862541Z",
  "updateTime": "2021-03-10T14:23:59.862541Z"
}
```


In [ ]:
# The fully qualified ID for the batch job
batch_job_id = request.name
# The short numeric ID for the batch job
batch_job_short_id = batch_job_id.split("/")[-1]

print(batch_job_id)

### [projects.locations.batchPredictionJobs.get](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.batchPredictionJobs/get)


#### Call


In [ ]:
request = clients["job"].get_batch_prediction_job(name=batch_job_id)

#### Response


In [ ]:
print(MessageToJson(request.__dict__["_pb"]))

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/batchPredictionJobs/6806214470445039616",
  "displayName": "traffic_20210310013516",
  "model": "projects/116273516712/locations/us-central1/models/6125898247828406272",
  "inputConfig": {
    "instancesFormat": "jsonl",
    "gcsSource": {
      "uris": [
        "gs://migration-ucaip-trainingaip-20210310013516/test.jsonl"
      ]
    }
  },
  "modelParameters": {
    "maxPredictions": 2.0,
    "confidenceThreshold": 0.5
  },
  "outputConfig": {
    "predictionsFormat": "jsonl",
    "gcsDestination": {
      "outputUriPrefix": "gs://migration-ucaip-trainingaip-20210310013516/batch_output/"
    }
  },
  "state": "JOB_STATE_RUNNING",
  "completionStats": {
    "incompleteCount": "2"
  },
  "createTime": "2021-03-10T14:23:59.862541Z",
  "startTime": "2021-03-10T14:24:00.012555Z",
  "updateTime": "2021-03-10T14:24:00.520535Z"
}
```


In [ ]:
def get_latest_predictions(gcs_out_dir):
    """ Get the latest prediction subfolder using the timestamp in the subfolder name"""
    folders = !gsutil ls $gcs_out_dir
    latest = ""
    for folder in folders:
        subfolder = folder.split("/")[-2]
        if subfolder.startswith("prediction-"):
            if subfolder > latest:
                latest = folder[:-1]
    return latest


while True:
    response = clients["job"].get_batch_prediction_job(name=batch_job_id)
    if response.state != aip.JobState.JOB_STATE_SUCCEEDED:
        print("The job has not completed:", response.state)
        if response.state == aip.JobState.JOB_STATE_FAILED:
            break
    else:
        folder = get_latest_predictions(
            response.output_config.gcs_destination.output_uri_prefix
        )
        ! gsutil ls $folder/prediction**

        ! gsutil cat $folder/prediction**
        break
    time.sleep(60)

*Example output*:
```
gs://migration-ucaip-trainingaip-20210310013516/batch_output/prediction-traffic_20210310013516-2021-03-10T14:23:59.650333Z/predictions_00001.jsonl
gs://migration-ucaip-trainingaip-20210310013516/batch_output/prediction-traffic_20210310013516-2021-03-10T14:23:59.650333Z/predictions_00002.jsonl
{"instance":{"content":"gs://automl-video-demo-data/traffic_videos/highway_005.mp4","mimeType":"video/avi","timeSegmentStart":"0.0s","timeSegmentEnd":"inf"},"prediction":[{"id":"6593913068772655104","displayName":"pickup_suv_van","timeSegmentStart":"35.300s","timeSegmentEnd":"35.900s","confidence":0.5733388,"frames":[{"timeOffset":"35.300s","xMin":0.92079705,"xMax":1.0,"yMin":0.52717,"yMax":0.6845663},{"timeOffset":"35.400s","xMin":0.86325824,"xMax":0.999999,"yMin":0.5247129,"yMax":0.68186224},{"timeOffset":"35.500s","xMin":0.79357177,"xMax":0.99050075,"yMin":0.5186033,"yMax":0.68388295},{"timeOffset":"35.600s","xMin":0.7312134,"xMax":0.935794,"yMin":0.5121129,"yMax":0.68021643},{"timeOffset":"35.700s","xMin":0.6609115,"xMax":0.8773811,"yMin":0.50215065,"yMax":0.6793843},{"timeOffset":"35.800s","xMin":0.593415,"xMax":0.816827,"yMin":0.4967009,"yMax":0.677144},{"timeOffset":"35.900s","xMin":0.51087815,"xMax":0.75138736,"yMin":0.4922755,"yMax":0.6732076}]},{"id":"6593913068772655104","displayName":"pickup_suv_van","timeSegmentStart":"46.900s","timeSegmentEnd":"47.900s","confidence":0.5721089,"frames":[{"timeOffset":"46.900s","xMin":0.8938238,"xMax":1.0,"yMin":0.53446406,"yMax":0.6901284},{"timeOffset":"47s","xMin":0.81357133,"xMax":0.99999857,"yMin":0.5293013,"yMax":0.6901114},{"timeOffset":"47.100s","xMin":0.733154,"xMax":0.94289106,"yMin":0.52340066,"yMax":0.69049513},{"timeOffset":"47.200s","xMin":0.6544491,"xMax":0.87583166,"yMin":0.51654726,"yMax":0.68932515},{"timeOffset":"47.300s","xMin":0.56814355,"xMax":0.7984497,"yMin":0.50629544,"yMax":0.6880638},{"timeOffset":"47.400s","xMin":0.47772846,"xMax":0.7148553,"yMin":0.49765483,"yMax":0.68850183},{"timeOffset":"47.500s","xMin":0.3756373,"xMax":0.6187881,"yMin":0.49258503,"yMax":0.68797386},{"timeOffset":"47.600s","xMin":0.28856453,"xMax":0.5317154,"yMin":0.48884195,"yMax":0.6842308},{"timeOffset":"47.700s","xMin":0.2014918,"xMax":0.44464266,"yMin":0.48509887,"yMax":0.6804877},{"timeOffset":"47.800s","xMin":0.11441906,"xMax":0.35756993,"yMin":0.48135576,"yMax":0.6767446},{"timeOffset":"47.900s","xMin":0.0075410376,"xMax":0.16627955,"yMin":0.4600201,"yMax":0.67653906}]},{"id":"6593913068772655104","displayName":"pickup_suv_van","timeSegmentStart":"30.700s","timeSegmentEnd":"31.800s","confidence":0.5617838,"frames":[{"timeOffset":"30.700s","xMin":0.9150882,"xMax":1.0,"yMin":0.5151331,"yMax":0.6645079},{"timeOffset":"30.800s","xMin":0.86258525,"xMax":0.99999905,"yMin":0.5172026,"yMax":0.6603533},{"timeOffset":"30.900s","xMin":0.801704,"xMax":0.9690958,"yMin":0.5114948,"yMax":0.65620536},{"timeOffset":"31s","xMin":0.7319551,"xMax":0.90261525,"yMin":0.5059969,"yMax":0.65359175},{"timeOffset":"31.100s","xMin":0.66124815,"xMax":0.83811176,"yMin":0.49920097,"yMax":0.65197486},{"timeOffset":"31.200s","xMin":0.5840557,"xMax":0.77017546,"yMin":0.49158275,"yMax":0.6492404},{"timeOffset":"31.300s","xMin":0.49507296,"xMax":0.7009768,"yMin":0.48103508,"yMax":0.6466039},{"timeOffset":"31.400s","xMin":0.405905,"xMax":0.61765563,"yMin":0.4749309,"yMax":0.6409255},{"timeOffset":"31.500s","xMin":0.3119387,"xMax":0.5226898,"yMin":0.46739954,"yMax":0.639586},{"timeOffset":"31.600s","xMin":0.21487714,"xMax":0.4266367,"yMin":0.46247935,"yMax":0.6337805},{"timeOffset":"31.700s","xMin":0.104759425,"xMax":0.32538062,"yMin":0.4524644,"yMax":0.63389504},{"timeOffset":"31.800s","xMin":0.018637476,"xMax":0.18262094,"yMin":0.438457,"yMax":0.6318268}]},{"id":"6593913068772655104","displayName":"pickup_suv_van","timeSegmentStart":"19.200s","timeSegmentEnd":"20.400s","confidence":0.5343286,"frames":[{"timeOffset":"19.200s","xMin":0.53855574,"xMax":0.7483454,"yMin":0.7698453,"yMax":0.88802785},{"timeOffset":"19.300s","xMin":0.7066706,"xMax":0.9046804,"yMin":0.5262296,"yMax":0.68281764},{"timeOffset":"19.400s","xMin":0.6643509,"xMax":0.8667256,"yMin":0.52155364,"yMax":0.683776},{"timeOffset":"19.500s","xMin":0.60929006,"xMax":0.81281793,"yMin":0.51661617,"yMax":0.6828631},{"timeOffset":"19.600s","xMin":0.5457967,"xMax":0.75832534,"yMin":0.51252514,"yMax":0.6830068},{"timeOffset":"19.700s","xMin":0.48583922,"xMax":0.7025621,"yMin":0.5050785,"yMax":0.6833887},{"timeOffset":"19.800s","xMin":0.42353436,"xMax":0.65267944,"yMin":0.499186,"yMax":0.68389857},{"timeOffset":"19.900s","xMin":0.36298347,"xMax":0.5871576,"yMin":0.49524042,"yMax":0.68120617},{"timeOffset":"20s","xMin":0.28549758,"xMax":0.51240987,"yMin":0.48895967,"yMax":0.6785187},{"timeOffset":"20.100s","xMin":0.20944653,"xMax":0.439814,"yMin":0.48409462,"yMax":0.6736581},{"timeOffset":"20.200s","xMin":0.12498511,"xMax":0.3544973,"yMin":0.47413808,"yMax":0.6721386},{"timeOffset":"20.300s","xMin":0.047281284,"xMax":0.26844072,"yMin":0.46798372,"yMax":0.66819036},{"timeOffset":"20.400s","xMin":-9.64848E-4,"xMax":0.1562836,"yMin":0.45929697,"yMax":0.66720545}]},{"id":"6593913068772655104","displayName":"pickup_suv_van","timeSegmentStart":"21.100s","timeSegmentEnd":"22.500s","confidence":0.5130946,"frames":[{"timeOffset":"21.100s","xMin":0.96134996,"xMax":0.9994743,"yMin":0.5534551,"yMax":0.69061935},{"timeOffset":"21.200s","xMin":0.70565313,"xMax":0.9051543,"yMin":0.52271163,"yMax":0.68131983},{"timeOffset":"21.300s","xMin":0.66035485,"xMax":0.8695388,"yMin":0.5142326,"yMax":0.68292105},{"timeOffset":"21.400s","xMin":0.60553956,"xMax":0.81872016,"yMin":0.5056492,"yMax":0.6834419},{"timeOffset":"21.500s","xMin":0.54476696,"xMax":0.7676598,"yMin":0.5011589,"yMax":0.6836144},{"timeOffset":"21.600s","xMin":0.48482427,"xMax":0.7100822,"yMin":0.49725318,"yMax":0.68293834},{"timeOffset":"21.700s","xMin":0.42285097,"xMax":0.65375054,"yMin":0.49218974,"yMax":0.6809865},{"timeOffset":"21.800s","xMin":0.35869005,"xMax":0.5910624,"yMin":0.48708224,"yMax":0.68020374},{"timeOffset":"21.900s","xMin":0.29066974,"xMax":0.52481556,"yMin":0.4808314,"yMax":0.68033195},{"timeOffset":"22s","xMin":0.22343048,"xMax":0.460608,"yMin":0.4752792,"yMax":0.68010074},{"timeOffset":"22.100s","xMin":0.15115453,"xMax":0.39462602,"yMin":0.46882644,"yMax":0.67740893},{"timeOffset":"22.200s","xMin":0.07956007,"xMax":0.32599604,"yMin":0.46620452,"yMax":0.6739611},{"timeOffset":"22.300s","xMin":0.019409377,"xMax":0.20294939,"yMin":0.4610349,"yMax":0.67052174},{"timeOffset":"22.400s","xMin":-0.017409453,"xMax":0.13916901,"yMin":0.45393682,"yMax":0.66754013},{"timeOffset":"22.500s","xMin":-0.02455799,"xMax":0.0710371,"yMin":0.45522687,"yMax":0.66398114}]},{"id":"6593913068772655104","displayName":"pickup_suv_van","timeSegmentStart":"7.200s","timeSegmentEnd":"7.800s","confidence":0.50423145,"frames":[{"timeOffset":"7.200s","xMin":0.77211607,"xMax":0.9202541,"yMin":0.55106527,"yMax":0.67469126},{"timeOffset":"7.300s","xMin":0.9063811,"xMax":0.9998785,"yMin":0.52542484,"yMax":0.68584275},{"timeOffset":"7.400s","xMin":0.8464132,"xMax":0.9988885,"yMin":0.5182979,"yMax":0.6811758},{"timeOffset":"7.500s","xMin":0.7691617,"xMax":0.9857696,"yMin":0.5142189,"yMax":0.6805917},{"timeOffset":"7.600s","xMin":0.7101507,"xMax":0.91866463,"yMin":0.50963223,"yMax":0.6773922},{"timeOffset":"7.700s","xMin":0.6297891,"xMax":0.85534894,"yMin":0.505215,"yMax":0.6743725},{"timeOffset":"7.800s","xMin":0.5598867,"xMax":0.7586217,"yMin":0.5045158,"yMax":0.67020833}]},{"id":"6593913068772655104","displayName":"pickup_suv_van","timeSegmentStart":"60.800s","timeSegmentEnd":"60.900s","confidence":0.5034977,"frames":[{"timeOffset":"60.800s","xMin":0.6538362,"xMax":0.88029855,"yMin":0.59181464,"yMax":0.769181},{"timeOffset":"60.900s","xMin":0.53221023,"xMax":0.7752164,"yMin":0.58824813,"yMax":0.77228796}]},{"id":"6593913068772655104","displayName":"pickup_suv_van","timeSegmentStart":"49.800s","timeSegmentEnd":"50s","confidence":0.5016138,"frames":[{"timeOffset":"49.800s","xMin":0.5714027,"xMax":0.8481284,"yMin":0.63446057,"yMax":0.88131857},{"timeOffset":"49.900s","xMin":0.3961952,"xMax":0.6958802,"yMin":0.6273483,"yMax":0.8836541},{"timeOffset":"50s","xMin":0.20160168,"xMax":0.50914085,"yMin":0.6205848,"yMax":0.89831626}]}]}
{"instance":{"content":"gs://automl-video-demo-data/traffic_videos/highway_006.mp4","mimeType":"video/avi","timeSegmentStart":"0.0s","timeSegmentEnd":"inf"},"prediction":[{"id":"6593913068772655104","displayName":"pickup_suv_van","timeSegmentStart":"33.600s","timeSegmentEnd":"34.300s","confidence":0.57430255,"frames":[{"timeOffset":"33.600s","xMin":0.7709672,"xMax":0.95710844,"yMin":0.50214607,"yMax":0.663173},{"timeOffset":"33.700s","xMin":0.6879625,"xMax":0.8816852,"yMin":0.5102545,"yMax":0.67594415},{"timeOffset":"33.800s","xMin":0.60038805,"xMax":0.7979881,"yMin":0.5179747,"yMax":0.6842574},{"timeOffset":"33.900s","xMin":0.49664575,"xMax":0.7060626,"yMin":0.5262368,"yMax":0.69601244},{"timeOffset":"34s","xMin":0.40653434,"xMax":0.62025917,"yMin":0.5326814,"yMax":0.7117975},{"timeOffset":"34.100s","xMin":0.29995015,"xMax":0.5285418,"yMin":0.53684795,"yMax":0.7238653},{"timeOffset":"34.200s","xMin":0.18591899,"xMax":0.42276734,"yMin":0.5479096,"yMax":0.7368871},{"timeOffset":"34.300s","xMin":0.06478273,"xMax":0.3098502,"yMin":0.5567089,"yMax":0.7530978}]},{"id":"8899756077986349056","displayName":"large_veh_bus","timeSegmentStart":"42.800s","timeSegmentEnd":"43.600s","confidence":0.5635853,"frames":[{"timeOffset":"42.800s","xMin":0.896148,"xMax":0.99750274,"yMin":0.08421734,"yMax":0.27437803},{"timeOffset":"42.900s","xMin":0.61661655,"xMax":0.9991679,"yMin":0.34556246,"yMax":0.7192955},{"timeOffset":"43s","xMin":0.5206372,"xMax":0.9830786,"yMin":0.35251692,"yMax":0.72906935},{"timeOffset":"43.100s","xMin":0.415886,"xMax":0.95419466,"yMin":0.3468655,"yMax":0.74923456},{"timeOffset":"43.200s","xMin":0.33259684,"xMax":0.89674544,"yMin":0.3457283,"yMax":0.7659639},{"timeOffset":"43.300s","xMin":0.24448554,"xMax":0.813682,"yMin":0.34896624,"yMax":0.77039707},{"timeOffset":"43.400s","xMin":0.14297022,"xMax":0.7258636,"yMin":0.34973124,"yMax":0.78382397},{"timeOffset":"43.500s","xMin":0.035293583,"xMax":0.6205815,"yMin":0.3503142,"yMax":0.797604},{"timeOffset":"43.600s","xMin":-0.012641478,"xMax":0.5095917,"yMin":0.34936112,"yMax":0.7985512}]},{"id":"6593913068772655104","displayName":"pickup_suv_van","timeSegmentStart":"14.300s","timeSegmentEnd":"14.800s","confidence":0.5230016,"frames":[{"timeOffset":"14.300s","xMin":0.6586052,"xMax":0.92509496,"yMin":0.56249046,"yMax":0.7750921},{"timeOffset":"14.400s","xMin":0.54897916,"xMax":0.83694077,"yMin":0.57280374,"yMax":0.7998729},{"timeOffset":"14.500s","xMin":0.42660663,"xMax":0.7059393,"yMin":0.5838014,"yMax":0.81943566},{"timeOffset":"14.600s","xMin":0.28328148,"xMax":0.5759178,"yMin":0.5971082,"yMax":0.8422871},{"timeOffset":"14.700s","xMin":0.14642228,"xMax":0.47787058,"yMin":0.60969377,"yMax":0.86163664},{"timeOffset":"14.800s","xMin":0.13050511,"xMax":0.393794,"yMin":0.6079176,"yMax":0.8719975}]},{"id":"8899756077986349056","displayName":"large_veh_bus","timeSegmentStart":"36.400s","timeSegmentEnd":"37.200s","confidence":0.5050303,"frames":[{"timeOffset":"36.400s","xMin":0.45120007,"xMax":0.6234123,"yMin":0.4991348,"yMax":0.68943614},{"timeOffset":"36.500s","xMin":0.52608997,"xMax":0.97820985,"yMin":0.37224957,"yMax":0.70904154},{"timeOffset":"36.600s","xMin":0.43248644,"xMax":0.9457052,"yMin":0.3685041,"yMax":0.7249851},{"timeOffset":"36.700s","xMin":0.32448632,"xMax":0.8723778,"yMin":0.3692387,"yMax":0.736981},{"timeOffset":"36.800s","xMin":0.24751942,"xMax":0.7811472,"yMin":0.36809424,"yMax":0.76570237},{"timeOffset":"36.900s","xMin":0.11482327,"xMax":0.6963299,"yMin":0.36990193,"yMax":0.7694417},{"timeOffset":"37s","xMin":0.014343479,"xMax":0.5851304,"yMin":0.37472367,"yMax":0.7721312},{"timeOffset":"37.100s","xMin":-0.008719128,"xMax":0.47485036,"yMin":0.37370107,"yMax":0.77210236},{"timeOffset":"37.200s","xMin":-0.0062835654,"xMax":0.35681728,"yMin":0.36186668,"yMax":0.811519}]},{"id":"6593913068772655104","displayName":"pickup_suv_van","timeSegmentStart":"30.700s","timeSegmentEnd":"31.300s","confidence":0.50222003,"frames":[{"timeOffset":"30.700s","xMin":0.5281936,"xMax":0.68637884,"yMin":0.52747375,"yMax":0.68398994},{"timeOffset":"30.800s","xMin":0.57842255,"xMax":0.78849596,"yMin":0.5235871,"yMax":0.6842843},{"timeOffset":"30.900s","xMin":0.4813683,"xMax":0.69776064,"yMin":0.5304373,"yMax":0.69869477},{"timeOffset":"31s","xMin":0.36337966,"xMax":0.5917747,"yMin":0.53825593,"yMax":0.7169143},{"timeOffset":"31.100s","xMin":0.23924252,"xMax":0.4805882,"yMin":0.53838265,"yMax":0.7325624},{"timeOffset":"31.200s","xMin":0.112733364,"xMax":0.37042505,"yMin":0.5456531,"yMax":0.7470921},{"timeOffset":"31.300s","xMin":0.021284297,"xMax":0.18838634,"yMin":0.5539143,"yMax":0.7607572}]},{"id":"6593913068772655104","displayName":"pickup_suv_van","timeSegmentStart":"22.400s","timeSegmentEnd":"23s","confidence":0.5017651,"frames":[{"timeOffset":"22.400s","xMin":0.8683299,"xMax":1.0,"yMin":0.56906176,"yMax":0.7567184},{"timeOffset":"22.500s","xMin":0.75663006,"xMax":0.9971202,"yMin":0.5708618,"yMax":0.77356946},{"timeOffset":"22.600s","xMin":0.65595376,"xMax":0.9007106,"yMin":0.582666,"yMax":0.7840566},{"timeOffset":"22.700s","xMin":0.54099375,"xMax":0.7999944,"yMin":0.5906615,"yMax":0.80249023},{"timeOffset":"22.800s","xMin":0.41166627,"xMax":0.68448514,"yMin":0.59567124,"yMax":0.8259323},{"timeOffset":"22.900s","xMin":0.2598535,"xMax":0.54294544,"yMin":0.6135365,"yMax":0.84419787},{"timeOffset":"23s","xMin":0.081104435,"xMax":0.38947436,"yMin":0.6304356,"yMax":0.8707452}]},{"id":"6593913068772655104","displayName":"pickup_suv_van","timeSegmentStart":"45.200s","timeSegmentEnd":"45.800s","confidence":0.5006547,"frames":[{"timeOffset":"45.200s","xMin":0.93331534,"xMax":0.99947244,"yMin":0.5425535,"yMax":0.7291488},{"timeOffset":"45.300s","xMin":0.85503596,"xMax":0.9993583,"yMin":0.5599785,"yMax":0.75284433},{"timeOffset":"45.400s","xMin":0.7337758,"xMax":0.9910556,"yMin":0.5646295,"yMax":0.7706001},{"timeOffset":"45.500s","xMin":0.6204056,"xMax":0.89361084,"yMin":0.5727049,"yMax":0.79307556},{"timeOffset":"45.600s","xMin":0.49371505,"xMax":0.795267,"yMin":0.58835214,"yMax":0.81640065},{"timeOffset":"45.700s","xMin":0.35610467,"xMax":0.6617675,"yMin":0.6014027,"yMax":0.8416757},{"timeOffset":"45.800s","xMin":0.1996267,"xMax":0.5096757,"yMin":0.6206687,"yMax":0.8625379}]}]}
```


# Cleaning up

To clean up all GCP resources used in this project, you can [delete the GCP
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial.


In [ ]:
delete_dataset = True
delete_model = True
delete_pipeline = True
delete_batchjob = True
delete_bucket = True

# Delete the dataset using the Vertex AI fully qualified identifier for the dataset
try:
    if delete_dataset:
        clients["dataset"].delete_dataset(name=dataset_id)
except Exception as e:
    print(e)

# Delete the model using the Vertex AI fully qualified identifier for the model
try:
    if delete_model:
        clients["model"].delete_model(name=model_id)
except Exception as e:
    print(e)

# Delete the training pipeline using the Vertex AI fully qualified identifier for the training pipeline
try:
    if delete_pipeline:
        clients["pipeline"].delete_training_pipeline(name=training_pipeline_id)
except Exception as e:
    print(e)

# Delete the batch job using the Vertex AI fully qualified identifier for the batch job
try:
    if delete_batchjob:
        clients["job"].delete_batch_prediction_job(name=batch_job_id)
except Exception as e:
    print(e)

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r gs://$BUCKET_NAME